In [1]:
# Required imports.
from py import calc_ubi
from py.loss_functions import loss_metrics
from py.optimize import optimize

import pandas as pd

In [2]:
AGE_CATEGORIES = ["senior", "child"]
DIS_CATEGORIES = ["dis_base", "dis_severe", "dis_enhanced"]
REGIONS = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]
categories = AGE_CATEGORIES + DIS_CATEGORIES + REGIONS

# Define bounds
AGE_BOUNDS = (40, 240)  # Child, working-age adult, senior.
DIS_BOUNDS = (0, 200)  # Base, severe, and enhanced disability UBI supplements.
GEO_BOUNDS = (-50, 50)  # Will be relative to a baseline geo.
# Skip adult which is calculated.
bounds = ([AGE_BOUNDS] * len(AGE_CATEGORIES) +
          [DIS_BOUNDS] * len(DIS_CATEGORIES) +
          [GEO_BOUNDS] * len(REGIONS))

input_dict = {category: bound for category, bound in zip(categories, bounds)}
print(input_dict)

{'senior': (40, 240), 'child': (40, 240), 'dis_base': (0, 200), 'dis_severe': (0, 200), 'dis_enhanced': (0, 200), 'NORTH_EAST': (-50, 50), 'NORTH_WEST': (-50, 50), 'YORKSHIRE': (-50, 50), 'EAST_MIDLANDS': (-50, 50), 'WEST_MIDLANDS': (-50, 50), 'EAST_OF_ENGLAND': (-50, 50), 'LONDON': (-50, 50), 'SOUTH_EAST': (-50, 50), 'SOUTH_WEST': (-50, 50), 'WALES': (-50, 50), 'SCOTLAND': (-50, 50), 'NORTHERN_IRELAND': (-50, 50)}


In [3]:
def opt(reform):
    return optimize(input_dict, "mean_pct_loss", reform,
                    verbose=False, seed=0,
                    # Reforms don't always improve upon one another with the
                    # default tolerance of 0.01.
                    tol=0.0001)

In [4]:
%%time
reform_1 = opt("reform_1")
print(reform_1[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.515000e-01
losses                1.401599e+11
mean_pct_loss         5.190000e-02
mean_pct_loss_pwd2    5.910000e-02
gini                  2.749000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0519 

Optimal solution:
 adult               124
senior              205
child                76
dis_base              0
dis_severe            0
dis_enhanced          0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.05193291339399926
 message: 'Optimization terminated successfully.'
    nfev: 3861
     nit: 14
 success: True
       x: array([205.22258067,  75.59287535,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.      

In [5]:
%%time
reform_2 = opt("reform_2")
print(reform_2[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.538000e-01
losses                1.424423e+11
mean_pct_loss         4.960000e-02
mean_pct_loss_pwd2    5.320000e-02
gini                  2.703000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0496 

Optimal solution:
 adult               119
senior              205
child                77
dis_base             99
dis_severe            0
dis_enhanced          0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
     fun: 0.04963382541450626
 message: 'Optimization terminated successfully.'
    nfev: 8706
     nit: 33
 success: True
       x: array([2.05346641e+02, 7.72628614e+01, 9.87098125e+01, 9.69306028e-02,
       1.46374744e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.000000

In [6]:
%%time
reform_3 = opt("reform_3")
print(reform_3[0])  # OptimizeResult.

Loss by all metrics:
 loser_share           4.540000e-01
losses                1.423893e+11
mean_pct_loss         4.960000e-02
mean_pct_loss_pwd2    5.300000e-02
gini                  2.703000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0496 

Optimal solution:
 adult               115
senior              202
child                73
dis_base            101
dis_severe            0
dis_enhanced          0
NORTH_EAST            8
NORTH_WEST            7
YORKSHIRE             0
EAST_MIDLANDS         4
WEST_MIDLANDS         5
EAST_OF_ENGLAND       2
LONDON                7
SOUTH_EAST            1
SOUTH_WEST            5
WALES                 3
SCOTLAND              4
NORTHERN_IRELAND      5
dtype: int64
     fun: 0.049555920733265306
 message: 'Optimization terminated successfully.'
    nfev: 24006
     nit: 93
 success: True
       x: array([ 2.06561447e+02,  7.76411263e+01,  1.00666665e+02,  2.64256976e-01,
        4.54431610e-01,  2.91271283e+00,  1.92939784e+00, -4.67953679e+00,
     

Check that iterations improve.

In [7]:
assert reform_2[0].fun < reform_1[0].fun, "Reform 2 doesn't improve on 1"
assert reform_3[0].fun < reform_2[0].fun, "Reform 3 doesn't improve on 2"

Construct DataFrame to export, with parameter and optimization values.

In [8]:
ubi_params = pd.DataFrame(reform_1[1]).T
ubi_params.loc[1] = reform_2[1]
ubi_params.loc[2] = reform_3[1]
ubi_params['mean_pct_loss'] = [reform_1[0].fun,
                               reform_2[0].fun,
                               reform_3[0].fun]
ubi_params

,adult,senior,child,dis_base,dis_severe,dis_enhanced,NORTH_EAST,NORTH_WEST,YORKSHIRE,EAST_MIDLANDS,WEST_MIDLANDS,EAST_OF_ENGLAND,LONDON,SOUTH_EAST,SOUTH_WEST,WALES,SCOTLAND,NORTHERN_IRELAND,mean_pct_loss
0,123.922887,205.222581,75.592875,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051933
1,119.170205,205.346641,77.262861,98.709813,0.096931,0.146375,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.049634
2,114.859461,201.881910,72.961590,100.666665,0.264257,0.454432,7.59225,6.608935,0.0,4.415393,4.954605,1.858716,6.585955,1.305033,5.041505,3.311826,4.483983,4.679537,0.049556


In [9]:
ubi_params.to_csv("optimal_params.csv", index=False)